In [ ]:
# this notebook is saved in GIT also.
# git remote add origin https://github.com/gitshiva/ML-AI-RPS.git

%load_ext tensorboard
import matplotlib
import datetime

#wget --no-check-certificate https://storage.googleapis.com/learning-datasets/rps.zip -o /tmp/rps.zip
#curl https://storage.googleapis.com/learning-datasets/rps.zip -o /tmp/rps.zip

#wget --no-check-certificate https://storage.googleapis.com/learning-datasets/rps-test-set.zip -o /tmp/rps-test-set.zip
# curl https://storage.googleapis.com/learning-datasets/rps-test-set.zip -o /tmp/rps-test-set.zip

# since we are dealing with OS files, os module is needed
import os
#since source files are zip, we need zipfile module to unzip to access data
import zipfile

#local_zip = '/tmp/rps.zip'
# open in readonly mode
#zip_ref = zipfile.ZipFile(local_zip, 'r')
# unzip in tmp dir itself
#zip_ref.extractall('/tmp/')
# close file
#zip_ref.close()

#local_zip = '/tmp/rps-test-set.zip'
#zip_ref = zipfile.ZipFile(local_zip, 'r')
#zip_ref.extractall('/tmp')
#zip_ref.close()

rock_dir = os.path.join('/tmp/rps/1rock')
paper_dir = os.path.join('/tmp/rps/paper')
scissors_dir = os.path.join('/tmp/rps/scissors')

print('total training rock images = ', len(os.listdir(rock_dir)))
print('total training paper images = ', len(os.listdir(paper_dir)))
print('total training scissors images = ', len(os.listdir(scissors_dir)))

rock_files = os.listdir(rock_dir)
print(rock_files[:10])

paper_files = os.listdir(paper_dir)
print(paper_files[:10])

scissors_files = os.listdir(scissors_dir)
print(scissors_files[:10])

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_rock = [os.path.join(rock_dir, fname)
             for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname)
              for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname)
                 for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
    img = mpimg.imread(img_path)
    plt.imshow(img)
    plt.axis('Off')
    plt.show()

# now we go on to process the images
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/tmp/rps"
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip = True,
    fill_mode='nearest')

VALIDATION_DIR = "/tmp/rps-test-set"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=63
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=63
)

model = tf.keras.models.Sequential([
    # 1st convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # 2nd convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 3rd convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 4th convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten before sending to Dense
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [ ]:
# clear out old logs from prior runs
!rm -rf ./logs/

# callbacks
# check for loss and early stop
myes01 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, restore_best_weights=True, verbose=1)

# callback for tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(train_generator, epochs=35, steps_per_epoch=20, validation_data=validation_generator, verbose=1, validation_steps=3, callbacks=[myes01, tensorboard_callback])
print ('Done model fitting.')

#model.save("rps.h5")
model.save('rps.keras')
print ('Done saving model. This cell is now complete')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
import numpy as np
from keras.preprocessing import image
input1 = open("rps/rwtest/randomag-scissors.jpg", "rb")
path = input1.name
print (path)
img = image.load_img(path, target_size=(150,150))
input1.close()
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

img = mpimg.imread(path)
plt.imshow(img)
plt.axis('Off')
plt.show()

images = np.vstack([x])
classes = model.predict(images, batch_size=1)
#print(path)
print('Rock-Paper-Scissors')
print(classes)



In [ ]:
%tensorboard --logdir logs/fit --bind_all